### Import Data from Pickle

In [ ]:
from utils.load_utils import get_exp_meta
import os
from utils.load_utils import import_brava_data
import matplotlib.pyplot as plt
import pickle

# EXPERIMENT META 
exp_code = "b1382"
exp_meta = get_exp_meta(exp_code)
desired_sampling_freq = 100

# JUPYTER
%matplotlib widget
%reload_ext autoreload
%autoreload 2

# FILES
working_dir = os.getcwd()
data_dir = working_dir + "/data"
datapath = data_dir + "/" + exp_code + ".txt"
picklepath = data_dir + '/' + exp_code + "_" + str(desired_sampling_freq) + "hz.pickle"

# IMPORT DATA
args = dict((k, exp_meta[k]) for k in ["t_start", "t_end", "ROIs", "LVDT_readjustments", "LVDT_method"] if k in exp_meta)
data = import_brava_data(datapath, picklepath, downsample_factor=(exp_meta["sampling_freq"] / desired_sampling_freq), **args)


# HAVE A LOOK
print("\n", data.head())
plt.close("all")
plt.plot(data["TIME"], data["ON BOARD LVDT"], color="lightgrey")
plt.plot(data["TIME"], data["LVDT NO JUMPS"])
plt.plot(data["TIME"], data["LVDT VELOCITY"])
plt.show()

### Identify and Remove Jumps

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import detrend, convolve, find_peaks
from findiff import FinDiff

# PARAMETERS
LVDT_readjustments = []
LVDT_method = "negativevelocitysensitive"

# IDENTIFY LVDT JUMPS
data["LVDT NO JUMPS"] = data.loc[:,"ON BOARD LVDT"]
freq = int(1/(data["TIME"][2] - data["TIME"][1]))
jumps = []
if LVDT_method == "sta/lta":
    print("...finding LVDT peaks...")
    rolling_window_length = 5*freq
    rolling_mean = convolve(data["LVDT NO JUMPS"], np.ones(rolling_window_length)/rolling_window_length, "same")
    lvdt_detrended = data["LVDT NO JUMPS"] - rolling_mean
    peaks, _ = find_peaks(lvdt_detrended, height=30, distance=4*freq)
    jumps = [(int(p-2*freq), int(p+freq)) for p in peaks]
elif LVDT_method == "clusters":
    print("...finding LVDT peaks...")
    peaks, _ = find_peaks(data["LVDT NO JUMPS"], width=(0.019*freq, 0.25*freq), prominence=1)
    troughs, _ = find_peaks(-data["LVDT NO JUMPS"], width=(0.019*freq, 0.25*freq), prominence=1)
    peaks = np.sort(np.concatenate((peaks, troughs)))
    i = 0
    while i < (len(peaks) - 1):
        peak = peaks[i]
        if (peaks[i+1] - peak) < freq:
            cluster = [p for p in peaks if peak <= p <= (peak+2*freq) ]
            jumps.append((int(cluster[0]-freq), int(cluster[-1]+0.5*freq)))
            i = i + len(cluster)
        i += 1
elif LVDT_method == "negativevelocity":
    dt = data["TIME"][2] - data["TIME"][1]
    d_dt = FinDiff(0, dt)
    vel = d_dt(data["LVDT NO JUMPS"])
    peaks, _ = find_peaks(vel, height=200)
    i = 0
    while i < len(peaks):
        cluster = [p for p in peaks if peaks[i] <= p <= (peaks[i]+3*freq) ]
        jumps.append((int(cluster[0]-freq), int(cluster[-1]+0.5*freq)))
        i += len(cluster)
elif LVDT_method == "negativevelocitysensitive":
    dt = data["TIME"][2] - data["TIME"][1]
    d_dt = FinDiff(0, dt)
    vel = d_dt(data["LVDT NO JUMPS"])
    peaks, _ = find_peaks(vel, height=60)
    i = 0
    while i < len(peaks):
        cluster = [p for p in peaks if peaks[i] <= p <= (peaks[i]+3*freq) ]
        jumps.append((int(cluster[0]-freq), int(cluster[-1]+0.5*freq)))
        i += len(cluster)


# REMOVE LVDT READJUSTMENTS
if LVDT_readjustments is not None:
    for readj in LVDT_readjustments:
        data.loc[data["TIME"].between(readj[0], readj[1]), "LVDT NO JUMPS"] = np.nan

# REMOVE LVDT JUMPS
for jump in jumps:
    if(jump[0] < 0):
        data.loc[0:jump[1], "LVDT NO JUMPS"] = np.nan
    else:
        data.loc[jump[0]:jump[1], "LVDT NO JUMPS"] = np.nan

# COMPUTE LVDT VELOCITY
dt = data["TIME"][2] - data["TIME"][1]
d_dt = FinDiff(0, dt)
data["LVDT VELOCITY"] = -d_dt(data["LVDT NO JUMPS"])


# PLOT RESULTS
vel = -d_dt(data["ON BOARD LVDT"])
plt.close("all")
plt.plot(data["TIME"], data["ON BOARD LVDT"], color="lightgrey")
plt.plot(data["TIME"], data["LVDT NO JUMPS"])
plt.plot(data["TIME"], vel)
plt.plot(data["TIME"][peaks], data["ON BOARD LVDT"][peaks], "x", color="orange")
plt.show()

In [ ]:
clusters = []
jumps = []
i = 0
while i < (len(peaks) - 2):
    peak = peaks[i]
    if (peaks[i+1] - peak) < freq:
        cluster = [p for p in peaks if peak <= p <= (peak+2*freq) ]
        clusters.append(cluster)
        jumps.append((cluster[0]-freq, cluster[-1]+freq))
        i = i + len(cluster)

    i += 1



# REMOVE JUMPS FROM DATA
for jump in jumps:
    if(jump[0] < 0):
        lvdt[0:jump[1]] = np.nan
    else:
        lvdt[jump[0]:jump[1]] = np.nan


# PLOT DATA
plt.figure()
plt.plot(lvdt, color="lightgrey")
# for cluster in clusters:
#     plt.plot(lvdt[cluster])
# plt.plot(lvdt[peaks], "x")

# plt.figure()
# plt.plot(time, lvdt_detrended)
# plt.plot(time[jumps], lvdt_detrended[jumps], "x")
plt.show()

In [ ]:
print(data["TIME"])

In [ ]:
from findiff import FinDiff
dt = 1/freq
d_dt = FinDiff(0, dt)
vel = -d_dt(lvdt)

plt.close("all")
plt.plot(vel)
plt.plot(lvdt)
plt.show()

In [ ]:
straights = []
prev_end = 0
for jump in jumps:
    jump_start = jump-2*freq
    jump_end = jump+2*freq

    if(jump_start > 0):
        straights.append((prev_end, jump_start))
    else:
        jump_start = -1
    prev_end = jump_end
    
    lvdt[jump_start+1:jump_end-1] = np.nan

for i, straight in enumerate(straights):
    lvdt_ = lvdt[straight[0]:straight[1]]
    # plt.plot(lvdt_)
    xp = np.linspace(straight[0], straight[1], len(lvdt_))
    p = np.polyfit(lvdt_, xp, 1)
    print(p)
    pfunc = np.poly1d(p)
    
    plt.plot(xp, pfunc(xp))

# plt.close("all")
# for i, peak in enumerate(peaks):
#     if(i > 0 and i < (len(peaks) - 1) and peak[0] > 0 and peak[1] < len(lvdt)):
#         lvdt_ =  lvdt[peak[0]:peak[1]]
#         lvdt[peak[0]:peak[1]] = np.nan
#         plt.plot(data["TIME"][peak[0]:peak[1]], lvdt_)
# plt.show()

# plt.close("all")
# plt.figure()
# plt.plot(data["TIME"], lvdt_detrended)
# plt.plot(data["TIME"][peaks_], lvdt_detrended[peaks_], "x")
# plt.figure()
# plt.plot(lvdt)
# plt.plot(jumps, lvdt[jumps], "x")
plt.show()